Une classification s'effectue de la façon suivante :

• - 1. Transformer toutes les variables en variables numériques.

• - 2. Transformer la base en format svmlib.

• - 3. Créer une Pipeline contenant :

    • La transformation de la variable label en catégorie.
    • La transformation des features catégorielles.
    • Un modèle de classification.
    • Un transformateur inverse de l'indexation pour les prédictions créées.
    
• - 4. Evaluer le modèle.

In [1]:
import findspark

In [2]:
findspark.init('/home/huy/spark-2.2.3-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [4]:
sc = SparkContext.getOrCreate()

In [5]:
spark = SparkSession.builder.appName('Pipelines').getOrCreate()
spark

In [6]:
from pyspark.ml.regression import LinearRegression

### Objectif: pouvoir appliquer tout algorithme de classification sur n'importe quelle base de donnée structurée

Les ML pipelines permettent d'enchaîner une succession d'estimateurs ou de transformeurs et à définir un processus de ML

Le format svmlib ne supporte pas les strings. L'indexeur permet de transformer une variable catégorielle en une série d'indices.

la base de données Human Ressources Analytics
(https://www.kaggle.com/jacksonchou/hr-data-for-analytics) contient des variables continues et des
variables catégorielles.

Cette base de données est relativement petite (14999 lignes), elle prend en compte di!érents indices de satisfaction et d'implication d'employés dans une société fictive. Ces données doivent permettre de déterminer quels employés sont susceptibles de quitter l'entreprise.

La variable à prédire est la variable left. Elle indique si l'employé a quitté la boîte volontairement ou non.

In [7]:
data = spark.read.csv("HR_comma_sep.csv",inferSchema=True,header=True)

In [8]:
data.printSchema()

root
 |-- satisfaction_level: double (nullable = true)
 |-- last_evaluation: double (nullable = true)
 |-- number_project: integer (nullable = true)
 |-- average_montly_hours: integer (nullable = true)
 |-- time_spend_company: integer (nullable = true)
 |-- Work_accident: integer (nullable = true)
 |-- left: integer (nullable = true)
 |-- promotion_last_5years: integer (nullable = true)
 |-- sales: string (nullable = true)
 |-- salary: string (nullable = true)



In [9]:
data.sample(False, 0.001, seed = 222).toPandas()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.54,0.77,4,271,3,0,0,0,support,medium
1,0.72,0.85,3,186,4,0,0,0,technical,low
2,0.57,0.65,5,177,2,0,0,0,IT,high
3,0.89,0.91,5,224,3,1,0,0,sales,low
4,0.83,0.72,4,161,3,0,0,0,hr,low
5,0.89,0.88,3,165,4,0,0,0,sales,medium
6,1.00,0.85,3,150,3,0,0,0,technical,low
7,0.81,0.69,5,109,2,0,0,0,IT,high
8,0.60,0.76,5,168,2,1,0,0,accounting,high
9,0.71,0.91,3,261,3,0,0,0,sales,low


In [10]:
data.columns

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

Placer la var left (le label que l'on veut prédire) en première colonne

In [11]:
data = data.select('left',
                    'satisfaction_level',
                    'last_evaluation',
                    'number_project',
                    'average_montly_hours',
                    'time_spend_company',
                    'Work_accident',
                    'promotion_last_5years',
                    'sales',
                    'salary')

data.describe().toPandas()

,summary,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,count,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999
1,mean,0.2380825388359224,0.6128335222348166,0.7161017401159978,3.80305353690246,201.0503366891126,3.498233215547703,0.1446096406427095,0.021268084538969265,None,None
2,stddev,0.42592409938029885,0.24863065106114257,0.17116911062327556,1.2325923553183513,49.94309937128406,1.4601362305354808,0.35171855238017957,0.1442814645785825,None,None
3,min,0,0.09,0.36,2,96,2,0,0,IT,high
4,max,1,1.0,1.0,7,310,10,1,1,technical,medium


Les colonnes sales et salary sont du type string, qui n'est pas géré par DenseVector.

Il faut utiliser la fonction StringIndexer pour transformer les données string en integer. Pour cela elle indexe les var en fonction de la fréquence. Le plus utilisé aura pour indice 0.

StringIndexer est un estimator. Elle s'utilise en 2 étapes:
- 1. créer l'indexeur en spécifiant les colonnes d'entrée et de sortie et chercher les modalités avec fit() ==> créer un modèle
- 2. appliquer l'indexeur avec transform()

In [12]:
from pyspark.ml.feature import StringIndexer

# créer un indexeur qui permet de transformer une variable en indice
salesIndexer = StringIndexer(inputCol='sales', outputCol='indexedSales').fit(data)

# applique l'indexeur et créer un dataframe
hrSalesIndexed = salesIndexer.transform(data)

hrSalesIndexed.sample(False, 0.001, seed = 222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSales
0,0,0.54,0.77,4,271,3,0,0,support,medium,2.0
1,0,0.72,0.85,3,186,4,0,0,technical,low,1.0
2,0,0.57,0.65,5,177,2,0,0,IT,high,3.0
3,0,0.89,0.91,5,224,3,1,0,sales,low,0.0
4,0,0.83,0.72,4,161,3,0,0,hr,low,8.0
5,0,0.89,0.88,3,165,4,0,0,sales,medium,0.0
6,0,1.00,0.85,3,150,3,0,0,technical,low,1.0
7,0,0.81,0.69,5,109,2,0,0,IT,high,3.0
8,0,0.60,0.76,5,168,2,1,0,accounting,high,7.0
9,0,0.71,0.91,3,261,3,0,0,sales,low,0.0


La fonction IndexToString est la fonction inverse de StringIndexer. A la différence qu'elle a gardé en mémoire les modalités indexées. Donc pas besoin de passer par l'étape fit()

In [13]:
from pyspark.ml.feature import IndexToString

# créer le transformator SalesRecontructor qui crée la colonne salesReconstructed 
# à partir du salesIndexer créée précédemment
SalesReconstructor = IndexToString(inputCol='indexedSales',
                                  outputCol='salesReconstructed',
                                  labels=salesIndexer.labels)

# appliquer le transformator sur le dataframe précédent
hrSalesReconstuctor = SalesReconstructor.transform(hrSalesIndexed)

hrSalesReconstuctor.sample(False, 0.001, seed = 222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSales,salesReconstructed
0,0,0.54,0.77,4,271,3,0,0,support,medium,2.0,support
1,0,0.72,0.85,3,186,4,0,0,technical,low,1.0,technical
2,0,0.57,0.65,5,177,2,0,0,IT,high,3.0,IT
3,0,0.89,0.91,5,224,3,1,0,sales,low,0.0,sales
4,0,0.83,0.72,4,161,3,0,0,hr,low,8.0,hr
5,0,0.89,0.88,3,165,4,0,0,sales,medium,0.0,sales
6,0,1.00,0.85,3,150,3,0,0,technical,low,1.0,technical
7,0,0.81,0.69,5,109,2,0,0,IT,high,3.0,IT
8,0,0.60,0.76,5,168,2,1,0,accounting,high,7.0,accounting
9,0,0.71,0.91,3,261,3,0,0,sales,low,0.0,sales


### Pipelines

Au lieu de passer par plusieurs étapes, pipeline permet d'enchaîner les étapes en une fois

Créer 2 indexeurs (des estimators) pour indexer les colonnes sales et salary

Créer une pipeline qui applique les deux indexeurs

Appliquer la pipeline sur le dataframe originel: data

In [14]:
from pyspark.ml import Pipeline

salesIndexer = StringIndexer(inputCol='sales', outputCol='indexedSales')
salaryIndexer = StringIndexer(inputCol='salary', outputCol='indexedSalary')

indexer = Pipeline(stages=[salesIndexer, salaryIndexer])

hrIndexed = indexer.fit(data).transform(data)

In [15]:
hrIndexed.sample(False, 0.001, seed = 222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSales,indexedSalary
0,0,0.54,0.77,4,271,3,0,0,support,medium,2.0,1.0
1,0,0.72,0.85,3,186,4,0,0,technical,low,1.0,0.0
2,0,0.57,0.65,5,177,2,0,0,IT,high,3.0,2.0
3,0,0.89,0.91,5,224,3,1,0,sales,low,0.0,0.0
4,0,0.83,0.72,4,161,3,0,0,hr,low,8.0,0.0
5,0,0.89,0.88,3,165,4,0,0,sales,medium,0.0,1.0
6,0,1.00,0.85,3,150,3,0,0,technical,low,1.0,0.0
7,0,0.81,0.69,5,109,2,0,0,IT,high,3.0,2.0
8,0,0.60,0.76,5,168,2,1,0,accounting,high,7.0,2.0
9,0,0.71,0.91,3,261,3,0,0,sales,low,0.0,0.0


In [16]:
hrIndexed.describe().toPandas()

,summary,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSales,indexedSalary
0,count,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999
1,mean,0.2380825388359224,0.6128335222348166,0.7161017401159978,3.80305353690246,201.0503366891126,3.498233215547703,0.1446096406427095,0.021268084538969265,None,None,2.69551303420228,0.5947063137542503
2,stddev,0.42592409938029885,0.24863065106114257,0.17116911062327556,1.2325923553183513,49.94309937128406,1.4601362305354808,0.35171855238017957,0.1442814645785825,None,None,2.754845263313967,0.6371829504695818
3,min,0,0.09,0.36,2,96,2,0,0,IT,high,0.0,0.0
4,max,1,1.0,1.0,7,310,10,1,1,technical,medium,9.0,2.0


In [17]:
hrIndexed.printSchema()

root
 |-- left: integer (nullable = true)
 |-- satisfaction_level: double (nullable = true)
 |-- last_evaluation: double (nullable = true)
 |-- number_project: integer (nullable = true)
 |-- average_montly_hours: integer (nullable = true)
 |-- time_spend_company: integer (nullable = true)
 |-- Work_accident: integer (nullable = true)
 |-- promotion_last_5years: integer (nullable = true)
 |-- sales: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- indexedSales: double (nullable = true)
 |-- indexedSalary: double (nullable = true)



## Exclure les colonnes non numériques avec de mettre la base en format svmlib

In [18]:
hrIndexed.columns

['left',
 'satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'sales',
 'salary',
 'indexedSales',
 'indexedSalary']

In [19]:
from pyspark.ml.linalg import DenseVector

hrNumeric = hrIndexed.select('left',
                             'satisfaction_level',
                             'last_evaluation',
                             'number_project',
                             'average_montly_hours',
                             'time_spend_company',
                             'Work_accident',
                             'promotion_last_5years',
                             'indexedSales',
                             'indexedSalary')

# passer par la structure RDD pour créer une variable DenseVector
# transformer chaque ligne en couple 1ère colonne, et le reste
hrRdd = hrNumeric.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# passer en format dataframe pour obtenir une base en format svmlib
# et nommer le colonnes
hrSvmlib = spark.createDataFrame(hrRdd, ['label', 'features'])

hrSvmlib.sample(False, 0.001, seed = 222).toPandas()

,label,features
0,0,"[0.54, 0.77, 4.0, 271.0, 3.0, 0.0, 0.0, 2.0, 1.0]"
1,0,"[0.72, 0.85, 3.0, 186.0, 4.0, 0.0, 0.0, 1.0, 0.0]"
2,0,"[0.57, 0.65, 5.0, 177.0, 2.0, 0.0, 0.0, 3.0, 2.0]"
3,0,"[0.89, 0.91, 5.0, 224.0, 3.0, 1.0, 0.0, 0.0, 0.0]"
4,0,"[0.83, 0.72, 4.0, 161.0, 3.0, 0.0, 0.0, 8.0, 0.0]"
5,0,"[0.89, 0.88, 3.0, 165.0, 4.0, 0.0, 0.0, 0.0, 1.0]"
6,0,"[1.0, 0.85, 3.0, 150.0, 3.0, 0.0, 0.0, 1.0, 0.0]"
7,0,"[0.81, 0.69, 5.0, 109.0, 2.0, 0.0, 0.0, 3.0, 2.0]"
8,0,"[0.6, 0.76, 5.0, 168.0, 2.0, 1.0, 0.0, 7.0, 2.0]"
9,0,"[0.71, 0.91, 3.0, 261.0, 3.0, 0.0, 0.0, 0.0, 0.0]"


In [21]:
# préciser que la variable label est catégorielle
# pour cela passer par l'estimator StringIndexer

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(hr_ml)

# pour les features, utiliser pour cela l'estimator VectorIndexer
# ce feature n'indexe que les variables qui ont au maximum de modalités
# ce seuil est spécifié par l'argument maxCategories
# ici, si une variable a plus de 5 modalités, alors elle sera considérée comme continue

featureIndexer = VectorIndexer(inputCol="features",
                                outputCol="indexedFeatures",
                                maxCategories = 5).fit(hr_ml)

# combien de variables de features sont catégorielles ?
hrNumeric.describe().toPandas()

# réponse 10 pour IndexedSales

,summary,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,indexedSales,indexedSalary
0,count,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999
1,mean,0.2380825388359224,0.6128335222348166,0.7161017401159978,3.80305353690246,201.0503366891126,3.498233215547703,0.1446096406427095,0.021268084538969265,2.69551303420228,0.5947063137542503
2,stddev,0.42592409938029885,0.24863065106114257,0.17116911062327556,1.2325923553183513,49.94309937128406,1.4601362305354808,0.35171855238017957,0.1442814645785825,2.754845263313967,0.6371829504695818
3,min,0,0.09,0.36,2,96,2,0,0,0.0,0.0
4,max,1,1.0,1.0,7,310,10,1,1,9.0,2.0


Limitation actuelle: 
il n'est pas possible d'indiquer précisemment les variables catégorielles autre que par le biais de maxCategories. 
en placant maxCategories à 10, l'algo considère que la variable numbr_project comme catégorielle.
en placant maxCategories à moins de 10, l'algo consière que la variable IndexedSales comme continue

## Application d'un classifieur ML

In [45]:
# application du classifieur (RandomForest)
# pour cela: indiquer que la varialble 'label' est catégorielle,
# et indiquer le seuil de modalité des variables 'features' au delà duquel les variables sont considérées
# comme continues

from pyspark.ml.feature import VectorIndexer
from pyspark.ml.classification import RandomForestClassifier

# créer une pipeline qui contient
#  des estimators labelIndexer et featureIndexer
#  un classifieur RandonForest qui précise les Indexer précédemment créés, et la colonne de prédiction
#  un transformateur IndexToString permettant de rétablir le label des prédictions

labelIndexer = StringIndexer(inputCol='label',
                             outputCol='indexedLabel').fit(hrSvmlib)

# indiquer le nombre max de modalités des variables catégorielles
featureIndexer = VectorIndexer(inputCol='features',
                               outputCol='indexedFeatures',
                               maxCategories=10).fit(hrSvmlib)

rf = RandomForestClassifier(labelCol='indexedLabel',
                            featuresCol='featureIndexer',
                            predictionCol='prediction',
                            seed=222)

labelConverter = IndexToString(inputCol='prediction',
                               outputCol='predictedLabel',
                               labels=labelIndexer.labels)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

In [46]:
(train, test) = hrSvmlib.randomSplit([0.7, 0.3], seed=222)

In [ ]:
# appliquer la pipeline à la base de donnée train
model = pipeline.fit(train)

In [ ]:
# appliquer le model entraîné avec train sur la base test

predictions = model.transform(test)

predictions.sample(False, 0.001, seed = 222).toPandas()

Vérifier la fiabilité des prédictions: comparer les prédictions à des valeurs réelles de test.

Et les comparer à d'autres modèles de classification.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName='accuracy',
                                             labelCol='indexedLabel',
                                             predictionCol='prediction')

accuracy = evaluator.evaluate(predictions)

print(accuracy)

0.9704168534289557

La métrique accuracy correspond au nombre de prédictions correctes divisé par le nombre de
prédictions e!ectuées. Elle est donc comprise entre 0 et 1 ; une accuracy de 0 correspond à des prédictions
toutes fausses et une accuracy de 1 correspond à l'absence d'erreur dans la prédiction.